In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import random
from datetime import datetime, timedelta
import pickle
from fuzzywuzzy import fuzz, process
from fractions import Fraction
import time
import collections
import concurrent.futures
from collections import defaultdict
import json

In [2]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [3]:
global oddschecker
oddschecker = 'https://www.oddschecker.com/football/'

In [191]:
def get_stats(url):
    driver = webdriver.Chrome()
    try:
        driver.implicitly_wait(random.randint(0, 6))
        driver.get(url)
        ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
        if ht != 'Half-Time':
            print('not halftime just yet ... sleeping')
            time.sleep(3*60)
            ht = driver.find_elements(By.XPATH, '//span[@class="css-8ngw4i-bottomRow"]')[1].text
            if ht != 'Half-Time':
                print(f'time: {ht}')
        button = driver.find_elements(By.XPATH, '(//button[@class="css-12l9xvx-ExpandButton e7i6a0l0"])[2]')[0]
        button.click()
        div1 = driver.find_elements(By.XPATH, '//li[@class="css-radwzz-Stat e683amr6"]')
        t = {k[1]: [k[0], k[2]] for k in [i.split('\n') for i in [x for x in [d.text for d in div1]
              if x not in ['Shots','Passes','Expected goals (xG)','Discipline','Defence','Duels']]]}
        score = driver.find_element(By.XPATH, '//span[@class="css-bw7eig-topRow"]').get_attribute('textContent').split(' ')
        t['score'] = [score[0], score[2]]
        posession = driver.find_element(By.XPATH, '//div[@class="css-7s52se-PossessionWheel e683amr3"]').text.split('\n')
        t['posession'] = [posession[1].replace('%',''), posession[2].replace('%','')]
        game_stats = {}
        for name, stats in t.items():
            if '(' in ''.join(stats):
                s = stats[0].split(' ') + stats[1].split(' ')
                if len(s) == 4:
                    s = [i.replace('(', '').replace(')','').replace('%','') for i in s]
                    game_stats[name] = [s[0], s[2]]
                    game_stats[name + ' percentage'] = [int(s[1])/100, int(s[3])/100]
                else:
                    s = [i for i in s if '%' not in i]
                    game_stats[name] = s
            else:
                game_stats[name] = stats
#         game_stats = {k: float(v[0]) - float(v[1]) for k, v in game_stats.items()}
        game_stats['match_name'] = url.split('/')[-1]
        game_stats['half_time'] = ht
        game_stats['url'] = url
    except:
        game_stats = {}
    driver.quit()
    return game_stats

In [58]:
get_stats('https://www.fotmob.com/match/3900943/matchfacts/aston-villa-vs-everton')

not halftime just yet ... sleeping


In [5]:
xg_leagues = {'UNITED STATES - MAJOR LEAGUE SOCCER':'world/usa/mls','ENGLAND - CHAMPIONSHIP':'english/championship',
              'ENGLAND - PREMIER LEAGUE': 'english/premier-league', 'GERMANY - BUNDESLIGA': 'germany/bundesliga',
              'FRANCE - LIGUE 1': 'france/ligue-1','AUSTRIA - BUNDESLIGA': 'austria/bundesliga',
              'PORTUGAL - LIGA PORTUGAL':'portugal/primeira-liga', 'BELGIUM - FIRST DIVISION A':'belgium/jupiler-pro-league',
              'SWITZERLAND - SUPER LEAGUE':'switzerland/super-league','SPAIN - LALIGA':'spain/la-liga-primera',
             'DENMARK - SUPERLIGAEN':'denmark/superligaen', 'MEXICO - LIGA MX APERTURA':'world/mexico/liga-mx',
             'NETHERLANDS - EREDIVISIE':'netherlands/eredivisie', 'SCOTLAND - PREMIERSHIP': 'scottish/premiership'}

def fotmob_game_data(url):
    driver = webdriver.Chrome()
    driver.implicitly_wait(3)
    driver.get(url)
    divs = driver.find_elements(By.XPATH, '//div[@class="css-aoqfcq-Group e7pc1841"]')
    today = url[-8:]
    league = {}
    for count, bucket in enumerate(divs):
        league_name = bucket.find_elements(By.XPATH, '//a[@class="css-vfisw6-GroupTitleLink e7pc1842"]')[count].text
        if league_name not in xg_leagues.keys():
            continue
        url = bucket.find_elements(By.TAG_NAME, 'a')
        urls = [u.get_attribute('href') for u in url]
        time = bucket.text
        time = [datetime.strptime(i + today, '%H:%M%Y%m%d') + timedelta(minutes=53) for i in time.split('\n') if ':' in i]
        urls = urls[1:]
        names = [u.split('/')[-1] for u in urls]
        league[league_name] = {n: [u,t] for u, t, n in zip(urls, time, names)}
    return league

In [187]:
game_data = fotmob_game_data('https://www.fotmob.com/?date=20220815')

In [8]:
def get_all_odds_links():
    odds_links = {}
    for f_league, url_league in xg_leagues.items():
        driver = webdriver.Chrome()
        driver.implicitly_wait(3)
        url = oddschecker + url_league
        driver.get(url)
        driver.implicitly_wait(3)
        divs = driver.find_elements(By.XPATH, '//a[@class="beta-callout full-height-link whole-row-link"]')
        mt = []
        for div in divs:
            odds_url = div.get_attribute('href')
            mt.append(odds_url)
        odds_links[f_league] = mt
    return odds_links

In [188]:
all_odds = get_all_odds_links()

In [10]:
def connect_links(data, all_odds):
    for l in data.keys():
        fotmob_names = data[l].keys()
        odds_names = {v: k for k, v in zip(all_odds[l],[str(i).split('/')[-2] for i in all_odds[l]])}
        for game in fotmob_names:
            match = getname(game, odds_names.keys())
            try:
                link = odds_names[match]
            except:
                link = ''
            data[l][game] = data[l][game] + [link]
    scraper_data = collections.defaultdict(list)
    for l_name, data in data.items():
        for game_name, l1_t_l2 in data.items():
            scraper_data[str(l1_t_l2[1])].append([game_name, l_name, l1_t_l2[0], l1_t_l2[2]])
    scraper_data = dict(sorted(scraper_data.items(), key=lambda x: x[0]))
    return scraper_data

In [189]:
days_data = connect_links(game_data, all_odds)

In [145]:
def get_odds(url):
    try:
        driver = webdriver.Chrome()
        if url == '':
            return {0:0,0:0,0:0}
        driver.implicitly_wait(1)
        url = url
        driver.get(url)
        odds = driver.find_element(By.XPATH, '//div[@class="ExpanderBestOddsWrapper_e1jc6hcf"]').text.split('\n')
        odds = [float(Fraction(j)) + 1.0 if '/' in j else j for j in odds]
        odds = dict(zip(odds[::2], odds[1::2]))
    except:
        odds = {0:0,0:0,0:0}
    driver.quit()
    return odds

In [84]:
get_odds('https://www.oddschecker.com/football/world/usa/mls/fc-cincinnati-v-atlanta-united/winner')

{'FC Cincinnati': 2.0, 'Draw': 4.0, 'Atlanta United': 3.6}

In [114]:
def stats_odds(stats_url, odds_url):
    stats = get_stats(stats_url)
    odds = get_odds(odds_url)
    row = {**stats, **odds}
    z.append(row)
    return row

In [192]:
days_data

{'2022-08-15 11:23:00': [['maritimo-vs-chaves',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937362/matchfacts/maritimo-vs-chaves',
   'https://www.oddschecker.com/football/portugal/primeira-liga/maritimo-v-chaves/winner']],
 '2022-08-15 12:23:00': [['athletic-club-vs-mallorca',
   'SPAIN - LALIGA',
   'https://www.fotmob.com/match/3918008/matchfacts/athletic-club-vs-mallorca',
   'https://www.oddschecker.com/football/spain/la-liga-primera/athletic-bilbao-v-mallorca/winner']],
 '2022-08-15 13:53:00': [['agf-aarhus-vs-lyngby',
   'DENMARK - SUPERLIGAEN',
   'https://www.fotmob.com/match/3887400/matchfacts/agf-aarhus-vs-lyngby',
   'https://www.oddschecker.com/football/denmark/superligaen/agf-aarhus-v-lyngby/winner'],
  ['arouca-vs-gil-vicente',
   'PORTUGAL - LIGA PORTUGAL',
   'https://www.fotmob.com/match/3937381/matchfacts/arouca-vs-gil-vicente',
   'https://www.oddschecker.com/football/portugal/primeira-liga/fc-arouca-v-gil-vicente/winner']],
 '2022-08-15 14:23:0

In [180]:
days_data = {
 '2022-08-14 20:58:00': [['tigres-vs-santos-laguna',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888525/matchfacts/tigres-vs-santos-laguna',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/tigres-v-club-santos-laguna/winner']],
 '2022-08-14 21:53:00': [['nashville-sc-vs-minnesota-united',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787604/matchfacts/nashville-sc-vs-minnesota-united',
   'https://www.oddschecker.com/football/world/usa/mls/nashville-sc-v-minnesota-united/winner']],
 '2022-08-14 22:53:00': [['seattle-sounders-fc-vs-real-salt-lake',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787587/matchfacts/seattle-sounders-fc-vs-real-salt-lake',
   'https://www.oddschecker.com/football/world/usa/mls/seattle-sounders-fc-v-real-salt-lake/winner']]}

In [181]:
c = days_data.copy()

In [182]:
days_data

{'2022-08-14 20:58:00': [['tigres-vs-santos-laguna',
   'MEXICO - LIGA MX APERTURA',
   'https://www.fotmob.com/match/3888525/matchfacts/tigres-vs-santos-laguna',
   'https://www.oddschecker.com/football/world/mexico/liga-mx/tigres-v-club-santos-laguna/winner']],
 '2022-08-14 21:53:00': [['nashville-sc-vs-minnesota-united',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787604/matchfacts/nashville-sc-vs-minnesota-united',
   'https://www.oddschecker.com/football/world/usa/mls/nashville-sc-v-minnesota-united/winner']],
 '2022-08-14 22:53:00': [['seattle-sounders-fc-vs-real-salt-lake',
   'UNITED STATES - MAJOR LEAGUE SOCCER',
   'https://www.fotmob.com/match/3787587/matchfacts/seattle-sounders-fc-vs-real-salt-lake',
   'https://www.oddschecker.com/football/world/usa/mls/seattle-sounders-fc-v-real-salt-lake/winner']]}

In [183]:
while True:
    if len(c) == 0:
        print('all done')
        break
    for clock in c.copy().keys():
        if datetime.strptime(clock, "%Y-%m-%d %H:%M:%S") <= datetime.now():
            stats_urls = [x[2] for x in c[clock]]
            odds_urls = [x[3] for x in c[clock]]
            print(stats_urls, odds_urls)
            with concurrent.futures.ThreadPoolExecutor() as executor:
                results = list(executor.map(stats_odds, stats_urls, odds_urls))
            del c[clock]
        else:
            results = []
    print(f'{len(c)} halftimes left -- {len(z)}')
    time.sleep(3 * 20)

3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes left -- 60
3 halftimes

In [184]:
z

[{'Expected goals (xG)': 0.37999999999999995,
  'Total shots': 2.0,
  'Big chances': 1.0,
  'Big chances missed': 1.0,
  'Accurate passes': 70.0,
  'Accurate passes percentage': 0.05999999999999994,
  'Fouls committed': 3.0,
  'Offsides': -1.0,
  'Corners': -1.0,
  'Shots off target': 2.0,
  'Shots on target': 2.0,
  'Blocked shots': -2.0,
  'Hit woodwork': 0.0,
  'Shots inside box': 2.0,
  'Shots outside box': 0.0,
  'xG open play': 0.43000000000000005,
  'xG set play': -0.05,
  'xG on target (xGOT)': 0.17,
  'Passes': 71.0,
  'Own half': 28.0,
  'Opposition half': 42.0,
  'Accurate long balls': 11.0,
  'Accurate long balls percentage': 0.26,
  'Accurate crosses': 4.0,
  'Accurate crosses percentage': 0.44,
  'Throws': -2.0,
  'Yellow cards': 0.0,
  'Red cards': 0.0,
  'Tackles won': 1.0,
  'Tackles won percentage': 0.0,
  'Interceptions': -3.0,
  'Blocks': 2.0,
  'Clearances': 5.0,
  'Keeper saves': -2.0,
  'Duels won': -3.0,
  'Ground duels won': -1.0,
  'Ground duels won percentage

In [193]:
open_file = open('data.pkl', "wb")
pickle.dump(z, open_file)
open_file.close()

In [194]:
open_file = open('data.pkl', "rb")
loaded_data = pickle.load(open_file)
open_file.close()

In [195]:
len(loaded_data)

63